# Reconnaissance des Digits basée sur les enregistrements Audios 

### Importer les bibliothéques necessaires 

In [123]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_style("whitegrid", {'axes.grid' : False})
sb.set(font_scale = 2)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from Tools.tools import rec
from Tools.tools import collection 
import warnings
warnings.filterwarnings('ignore')

### Collection 

In [124]:
# collection()

#### 1. Importer votre DataSet

In [125]:
mydata = pd.read_csv('./DataSet/'+os.listdir('./DataSet/')[0])

##### Print

In [126]:
mydata.head(20)

,Fe1,Fe2,Fe3,Fe4,Fe5,Fe6,Fe7,Fe8,Fe9,Fe10,Fe11,Fe12,Target
0,13.826166,-12.684228,12.645834,6.938258,1.434168,3.873297,1.214596,2.131574,4.101266,4.977142,2.475803,2.666064,0.0
1,14.423011,-13.687151,13.637553,2.500229,1.570457,3.071398,3.836199,4.864888,5.841415,6.643054,2.968906,1.927965,1.0
2,12.969076,-9.292158,13.908437,5.476703,4.484677,8.820590,8.712636,4.343233,2.952837,3.957840,1.200703,0.791016,2.0
3,12.585090,-11.316209,12.984019,4.367402,0.768229,0.558766,-0.600545,2.950509,3.842734,8.261035,3.326267,1.615859,3.0
4,13.630192,-13.826125,10.865906,1.812815,1.525143,5.304581,3.593793,4.086091,3.027385,4.901214,4.030547,3.319416,4.0
5,15.263614,-18.490760,7.125853,3.445330,-1.260309,4.997331,4.064654,3.001202,3.631085,5.674426,4.059043,1.290322,5.0
6,16.368568,-20.258037,4.861707,12.059828,-3.057030,2.082351,8.191053,4.652822,1.248023,5.294618,3.988588,0.432981,6.0
7,14.530073,-16.321594,9.120055,4.538796,-1.492503,4.744734,4.865404,1.646004,1.718095,3.987022,2.968077,2.558176,7.0
8,14.925852,-13.806911,11.038254,4.622788,2.752395,6.956544,4.885531,2.292389,2.405471,3.449352,4.409858,1.829354,8.0
9,15.152976,-13.615659,11.378413,2.599742,3.342093,8.825043,4.248628,5.147598,3.890636,4.679820,2.849187,2.540623,9.0


#### 2. Checker les variables quantitatives/qualitatives et les valeurs manquantes 

In [127]:
mydata.isna().value_counts()

Fe1    Fe2    Fe3    Fe4    Fe5    Fe6    Fe7    Fe8    Fe9    Fe10   Fe11   Fe12   Target
False  False  False  False  False  False  False  False  False  False  False  False  False     210
dtype: int64

#### 3. Visualiser les targets

In [128]:
mydata.Target

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
205    5.0
206    6.0
207    7.0
208    8.0
209    9.0
Name: Target, Length: 210, dtype: float64

#### 4. Notre variable target (Y) est 'gender', Récuprer X et y à partir du jeu de données 

In [129]:
y = mydata['Target']
X = mydata.iloc[:,:-1]
X.shape

(210, 12)

#### 5. Diviser la DataSet en donneés d'apprentissage et de test (20% pour le test)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

#### 6. Appliquer une normalisation centrée-réduite aux données en utilisant "StandardScaler"

In [131]:
sc=StandardScaler()
X_train_s = sc.fit_transform(X_train)
X_test_s = sc.fit_transform(X_test)
y_train_s = sc.fit_transform(np.array(y_train).reshape(-1,1))
y_test_s=sc.fit_transform(np.array(y_test).reshape(-1,1))

#### 7. Développer votre meilleur modèle de classification

In [132]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

svc = SVC()
dt = DecisionTreeClassifier()
rf= RandomForestClassifier()
gbc = GradientBoostingClassifier()
mlpc= MLPClassifier()
xgb = XGBClassifier()



In [ ]:

# pipe = Pipeline([('scaler', StandardScaler()),
#                  ('svc', SVC())])


pipe = Pipeline([('svc', SVC())])


parameters = { 'svc__kernel': ['linear', 'rbf'],
               'svc__C':[0.1,1,10,20]}

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.9163992869875223
{'svc__C': 10, 'svc__kernel': 'rbf'}
Score sur l'échantillon test 0.8571428571428571


In [ ]:

pipe = Pipeline([('scaler', StandardScaler()),
                 ('rf', RandomForestClassifier())])

parameters = { 'rf__n_estimators': [100,150,200,300]
               }

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.838680926916221
{'rf__n_estimators': 200}
Score sur l'échantillon test 0.7619047619047619


In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('dt', DecisionTreeClassifier())])

parameters = { 
               }

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.6122994652406417
{}
Score sur l'échantillon test 0.5714285714285714


In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('mlpc', MLPClassifier())])

parameters = {'mlpc__alpha' : [1e-2,1e-1,1,5,8], 'mlpc__hidden_layer_sizes' : [(50,50,50,10)]
                }

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.8631016042780748
{'mlpc__alpha': 0.01, 'mlpc__hidden_layer_sizes': (50, 50, 50, 10)}
Score sur l'échantillon test 0.8333333333333334


In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('gbc', GradientBoostingClassifier())])

parameters = { 'gbc__n_estimators' : [1,10,50,100,150,200]
               }

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.6718360071301247
{'gbc__n_estimators': 150}
Score sur l'échantillon test 0.6428571428571429


In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('xgb', XGBClassifier())])

parameters = { 'xgb__max_depth' : [9], 'xgb__learning_rate' : [ 0.1, 0.01, 0.001], 'xgb__n_estimators' : [100],
                'xgb__verbosity' : [0]
               }

Exp = GridSearchCV(pipe, 
                   param_grid = parameters, 
                   cv = 5)

Exp.fit(X_train, y_train)
print(Exp.best_score_)
print(Exp.best_params_)
print('Score sur l\'échantillon test', Exp.score(X_test,y_test))

0.7253119429590018
{'xgb__learning_rate': 0.1, 'xgb__max_depth': 9, 'xgb__n_estimators': 100, 'xgb__verbosity': 0}
Score sur l'échantillon test 0.6666666666666666


In [ ]:
import seaborn as sns
# sns.pairplot(mydata, hue='Target', corner = True)
#feature selection: S.fit(X_train,y_train)
# X_train =S.transform(X_train,X_test)

Le meilleur modèle dans notre cas est le SVC. J'ai supprimé le StandardScaler, qui faisait baisser le score.

Le score sur l'entrainement ressorti par le gridsearch est le suivant :
0.9163992869875223

Les paramètres optimaux sont les suivants :
{'svc__C': 10, 'svc__kernel': 'rbf'}

Score sur l'échantillon test 0.8571428571428571

# Application en Temps Réel

In [ ]:
from Tools.tools import rec
from Tools.tools import collection

In [146]:
opti=SVC(C=10, kernel = 'rbf').fit(X_train,y_train)
scaler=StandardScaler().fit(X_train,y_train)
rec(scaler,opti)

Attention, l'enregistrement commence dans :
5
4
3
2
1
0
Prononcer votre Digit : 
------------------
Digit :  2.0
------------------


In [147]:
from joblib import dump, load
dump(opti,'best_classifier')

['best_classifier']

In [148]:
model=load('best_classifier')